In [2]:
#pip install gspread oauth2client

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_earned = client.open("frb-volley-game-stats").worksheet("player-earned")

# Fetch all records (rows) from the sheet
player_earned_data = player_earned.get_all_records()

# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_earned_data)

In [2]:
df

,date,match-id,match,set,player,ace,spike,tip,dump,block,kills,attack-earned,total-earned
0,08/12/2024,141108,frb-gentofte,1,Lasse,1,0,0,1,0,1,1,2
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0,0,0,0
2,08/12/2024,141108,frb-gentofte,1,Kristian,0,1,0,0,0,1,1,1
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0,0,0,0
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,22/02/2025,141084,frb-holte,3,Mads,0,0,0,0,0,0,0,0
424,22/02/2025,141084,frb-holte,3,Martin,0,0,0,0,0,0,0,0
425,22/02/2025,141084,frb-holte,3,Mikkel,0,0,0,0,0,0,0,0
426,22/02/2025,141084,frb-holte,3,Frederik,0,0,0,0,0,0,0,0


In [3]:
points_per_set = df.groupby(['player', 'match', 'set'])['total-earned'].sum().reset_index()
points_per_set

,player,match,set,total-earned
0,Alex,avk-frb,1,0
1,Alex,avk-frb,2,0
2,Alex,avk-frb,3,0
3,Alex,avk-frb,4,0
4,Alex,avk-frb,5,0
...,...,...,...,...
422,Vestbjerg,gentofte-frb,1,2
423,Vestbjerg,gentofte-frb,2,1
424,Vestbjerg,gentofte-frb,3,5
425,Vestbjerg,gentofte-frb,4,0


In [4]:
points_per_game = df.groupby(['player', 'match'])['total-earned'].sum().reset_index()
points_per_game

,player,match,total-earned
0,Alex,avk-frb,0
1,Alex,frb-dtu,0
2,Alex,frb-gentofte,0
3,Alex,frb-holte,0
4,Alex,frb-kv61,0
...,...,...,...
99,Vestbjerg,frb-gentofte,3
100,Vestbjerg,frb-holte,0
101,Vestbjerg,frb-kv61,6
102,Vestbjerg,frb-vli,1


In [5]:
total_points = df.groupby('player')['total-earned'].sum().reset_index()
total_points

,player,total-earned
0,Alex,0
1,Ando,0
2,Bo,35
3,Boerme,52
4,Bosse,4
5,Frederik,2
6,Gustav,49
7,Kristian,22
8,Lasse,42
9,Mads,5


In [6]:
df_final = points_per_set.merge(points_per_game, on=['player', 'match'], suffixes=('_per_set', '_per_game'))
df_final = df_final.merge(total_points, on='player', suffixes=('', '_total'))
df_final

,player,match,set,total-earned_per_set,total-earned_per_game,total-earned
0,Alex,avk-frb,1,0,0,0
1,Alex,avk-frb,2,0,0,0
2,Alex,avk-frb,3,0,0,0
3,Alex,avk-frb,4,0,0,0
4,Alex,avk-frb,5,0,0,0
...,...,...,...,...,...,...
422,Vestbjerg,gentofte-frb,1,2,8,30
423,Vestbjerg,gentofte-frb,2,1,8,30
424,Vestbjerg,gentofte-frb,3,5,8,30
425,Vestbjerg,gentofte-frb,4,0,8,30


In [7]:
df_final.drop(columns='match', inplace=True)
df_final

,player,set,total-earned_per_set,total-earned_per_game,total-earned
0,Alex,1,0,0,0
1,Alex,2,0,0,0
2,Alex,3,0,0,0
3,Alex,4,0,0,0
4,Alex,5,0,0,0
...,...,...,...,...,...
422,Vestbjerg,1,2,8,30
423,Vestbjerg,2,1,8,30
424,Vestbjerg,3,5,8,30
425,Vestbjerg,4,0,8,30


In [8]:
total_points['points'] = total_points['total-earned']
total_points

,player,total-earned,points
0,Alex,0,0
1,Ando,0,0
2,Bo,35,35
3,Boerme,52,52
4,Bosse,4,4
5,Frederik,2,2
6,Gustav,49,49
7,Kristian,22,22
8,Lasse,42,42
9,Mads,5,5


In [9]:
points_per_game['points'] = points_per_game['total-earned']
points_per_game['max-earned'] = points_per_game.groupby('player')['points'].transform('sum')
points_per_game

,player,match,total-earned,points,max-earned
0,Alex,avk-frb,0,0,0
1,Alex,frb-dtu,0,0,0
2,Alex,frb-gentofte,0,0,0
3,Alex,frb-holte,0,0,0
4,Alex,frb-kv61,0,0,0
...,...,...,...,...,...
99,Vestbjerg,frb-gentofte,3,3,30
100,Vestbjerg,frb-holte,0,0,30
101,Vestbjerg,frb-kv61,6,6,30
102,Vestbjerg,frb-vli,1,1,30


In [10]:
import json

In [11]:
# Convert DataFrame to JSON
data = total_points.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../top-scorer.json", "w") as f:
    json.dump(data, f, indent=4)

#total_points.to_csv("../all_seasons_frb.csv", index=False)

In [12]:
# Convert DataFrame to JSON
data = points_per_game.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../top-scorer-per-game.json", "w") as f:
    json.dump(data, f, indent=4)

#total_points.to_csv("../all_seasons_frb.csv", index=False)